In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm  # Progress bar for training

In [2]:
class FERDataset(Dataset):
    def __init__(self, csv_file, usage, transform=None):
        self.data = pd.read_csv(csv_file)

        # Filter by usage (e.g., 'Training', 'PublicTest', 'PrivateTest')
        self.data = self.data[self.data[' Usage'] == usage]

        # Ensure valid pixel data (48x48 images)
        self.data = self.data[self.data[' pixels'].apply(lambda x: len(x.split()) == 2304)]
        
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixel_values = self.data.iloc[idx, 2].split()
        pixels = np.array(pixel_values, dtype=np.uint8).reshape(48, 48)
        image = Image.fromarray(pixels).convert('RGB')

        label = int(self.data.iloc[idx, 0])  # Emotion label
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [3]:
print("CUDA Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

CUDA Available: True
Number of GPUs: 1
GPU Name: Tesla P100-PCIE-16GB


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# 2. Config & Hyperparameters

csv_file_path = '/kaggle/input/fer-ck-augmentedfer/ferckaugmentedfer.csv'  # Update with your dataset path
num_classes = 7  # Number of emotion classes
batch_size = 32
num_epochs = 30
initial_lr = 0.001  # Start with 0.001 learning rate
decay_epochs = [5, 10, 15]  # Epochs where LR will be reduced
decay_factor = 0.5  # Reduce LR by this factor

In [6]:
# 3. Transforms & Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for SqueezeNet input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets
train_dataset = FERDataset(csv_file=csv_file_path, usage='Training', transform=transform)
test_dataset  = FERDataset(csv_file=csv_file_path, usage='PublicTest', transform=transform)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 67891
Test set size: 3589


In [7]:
# 4. Define the SqueezeNet Model
model = models.squeezenet1_1(pretrained=True)  # Load pretrained SqueezeNet

# Modify the classifier for emotion recognition
model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
model.num_classes = num_classes

model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 86.0MB/s]


In [8]:
# -----------------------------
# 5. Define Loss & Optimizer
# -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=initial_lr)

In [9]:
# -----------------------------
# 6. Function for Step Decay
# -----------------------------
def adjust_learning_rate(optimizer, epoch):
    """Manually adjusts the learning rate at specific epochs."""
    if epoch in decay_epochs:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= decay_factor  # Reduce LR by factor
        print(f"Learning rate adjusted at epoch {epoch} to {param_group['lr']}")

In [10]:
# -----------------------------
# 7. Training & Validation Loop
# -----------------------------
for epoch in range(num_epochs):
    adjust_learning_rate(optimizer, epoch)  # Adjust LR at specific epochs

    # --- Training ---
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] - Training")
    for images, labels in train_pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100.0 * correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%")

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    test_pbar = tqdm(test_loader, desc="Validation")
    with torch.no_grad():
        for images, labels in test_pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss_avg = val_loss / len(test_loader.dataset)
    val_acc = 100.0 * val_correct / val_total
    print(f"Val Loss: {val_loss_avg:.4f}, Val Acc: {val_acc:.2f}%")

    # Save checkpoint
    checkpoint_path = f"checkpoint_epoch_{epoch+1}.pth"
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': epoch_loss,
        'val_loss': val_loss_avg,
        'train_acc': epoch_acc,
        'val_acc': val_acc
    }, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")


Epoch [1/30] - Training: 100%|██████████| 2122/2122 [01:26<00:00, 24.47it/s]


Train Loss: 1.6980, Train Acc: 31.77%


Validation: 100%|██████████| 113/113 [00:04<00:00, 24.80it/s]


Val Loss: 1.4647, Val Acc: 42.57%
Checkpoint saved: checkpoint_epoch_1.pth


Epoch [2/30] - Training: 100%|██████████| 2122/2122 [01:24<00:00, 25.06it/s]


...

Val Loss: 0.1107, Val Acc: 96.54%
Checkpoint saved: checkpoint_epoch_29.pth


Epoch [30/30] - Training: 100%|██████████| 2122/2122 [01:24<00:00, 25.10it/s]


Train Loss: 0.0263, Train Acc: 99.25%


Validation: 100%|██████████| 113/113 [00:04<00:00, 27.41it/s]

Val Loss: 0.1465, Val Acc: 95.37%
Checkpoint saved: checkpoint_epoch_30.pth


In [11]:
# -----------------------------
# 8. Save the Final Model
# -----------------------------
torch.save(model.state_dict(), "squeezenet_fer_final.pth")
print("Training complete. Final model saved to squeezenet_fer_final.pth")

Training complete. Final model saved to squeezenet_fer_final.pth
